In [ ]:
import urllib.parse
import pandas as pd
import pymongo
import datetime

In [ ]:
MARKET_CODE_DICT = {
    'kospi': 'stockMkt',
    'kosdaq': 'kosdaqMkt',
    'konex': 'konexMkt'
}

DOWNLOAD_URL = 'kind.krx.co.kr/corpgeneral/corpList.do'

def download_stock_codes(market=None, delisted=False):
    
    params = {'method': 'download'}

    if market.lower() in MARKET_CODE_DICT:
        params['marketType'] = MARKET_CODE_DICT[market]

    if not delisted:
        params['searchType'] = 13

    params_string = urllib.parse.urlencode(params)
    request_url = urllib.parse.urlunsplit(['http', DOWNLOAD_URL, '', params_string, ''])

    df = pd.read_html(request_url, header=0)[0]
    df.종목코드 = df.종목코드.map('{:06d}'.format)
    
    df = df.rename(index=str, 
              columns={'회사명':'company', 
                       '종목코드':'code', 
                       '업종':'category',
                       '주요제품':'main_product', 
                       '상장일':'listed_date', 
                       '결산월':'settlement_month',
                       '대표자명':'representative',
                       '홈페이지':'homepage',
                       '지역':'reigon'})

    return df

In [ ]:
kospi_stocks = download_stock_codes('kospi')
kospi_stocks['suffix'] = 'KS'
kospi_stocks['_id'] = kospi_stocks['code']
kospi_stocks['update_date'] = datetime.datetime.now()
kosdaq_stocks = download_stock_codes('kosdaq')
kosdaq_stocks['suffix'] = 'KQ'
kosdaq_stocks['_id'] = kosdaq_stocks['code']
kosdaq_stocks['update_date'] = datetime.datetime.now()

In [ ]:
_MONGO_USER='root'
_MONGO_PASSWORD='example'
_MONGO_HOST='docker.for.mac.localhost'
_MONGO_PORT='27017'
_MONGO_URI='mongodb://{}:{}@{}:{}'.format(_MONGO_USER, _MONGO_PASSWORD, _MONGO_HOST, _MONGO_PORT)

In [ ]:
mongo_conn = pymongo.MongoClient(_MONGO_URI)
db = mongo_conn.get_database('money')
collection = db.get_collection('stock-code-new')

In [ ]:
collection.insert_many(kospi_stocks.to_dict('records'))

In [ ]:
collection.insert_many(kosdaq_stocks.to_dict('records'))

In [ ]:
ticker_kospi = kospi_stocks['code'].apply(lambda code: '{}.{}'.format(code, 'KS')).tolist()
ticker_kosdaq = kosdaq_stocks['code'].apply(lambda code: '{}.{}'.format(code, 'KQ')).tolist()

In [ ]:
def chunks(l, n):
    # For item i in a range that is a length of l,
    for i in range(0, len(l), n):
        # Create an index range for l of n items:
        yield l[i:i+n]

In [ ]:
chunked_kospi = list(chunks(ticker_kospi, 10))
chunked_kosdaq = list(chunks(ticker_kosdaq, 100))

In [ ]:
chunked_kosdaq[0]

In [ ]:
from pandas_datareader import data
import fix_yahoo_finance as yf
yf.pdr_override()

In [ ]:
def save_stock_price(tickers, start_date="1996-05-06"):
    
    price_collection = db["stock-price-new"]

    tot_cnt = len(tickers)
    print('total ticker count is {}'.format(tot_cnt))
    
    count = 0
    
    for ticker in tickers:
        stock_price = data.get_data_yahoo(tickers=ticker, start=start_date, thread=10)
        print(stock_price.columns.levels[0])
        adj_stock_price = stock_price.stack().reset_index(level=[0,1])
            
        final_df = pd.DataFrame()
        final_df['Date'] = pd.to_datetime(adj_stock_price['Date'])
        final_df['Code'] = adj_stock_price['level_1'].astype('str')
        final_df['Adj Close'] = pd.to_numeric(adj_stock_price['Adj Close'])
        final_df['Close'] = pd.to_numeric(adj_stock_price['Close'])
        final_df['High'] = pd.to_numeric(adj_stock_price['High'])
        final_df['Low'] = pd.to_numeric(adj_stock_price['Low'])
        final_df['Open'] = pd.to_numeric(adj_stock_price['Open'])
        final_df['Volume'] = pd.to_numeric(adj_stock_price['Volume'])
        final_df['date_str'] = pd.to_datetime(adj_stock_price['Date'], format='%Y%m%d').astype('str')
        final_df['_id'] = final_df[['Code', 'date_str']].apply(lambda row : "{}.{}".format(row[0], row[1]), axis=1)
        final_df = final_df.drop(columns=['date_str'])
        print(final_df.count())
        price_collection.insert_many(final_df.to_dict('records'))
        final_df = final_df[0:0]
        count += 1
        print("ticker count is {}".format(count*100))

In [ ]:
save_stock_price(chunked_kospi)

In [ ]:
save_stock_price(chunked_kosdaq)